In [1]:
from typing import Callable, Tuple
from finite_distributions.FiniteDistribution import FiniteDistribution
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sinkhorn.SinkhornRunner as SinkhornRunner
import sinkhorn.SinkhornKernels as skern
import visualizer.joint_distribution_visualizer as jdv

from core.require import require

from PIL import Image, ImageDraw

In [2]:
# Load the image (convert to grayscale or keep RGB depending on your need)
img_1 = Image.open("images/white_circle_on_black.png").convert("L")  # Use "RGB" if you want color
img_2 = Image.open("images/white_square_on_black.png").convert("L")  # Use "RGB" if you want color

# Convert to NumPy array
img_1_array = np.array(img_1)/255
img_2_array = np.array(img_2)/255

require(img_1_array.shape == img_2_array.shape)
require(len(img_1_array.shape) == 2)

rows = img_1_array.shape[0]
cols = img_1_array.shape[1]
require(rows == cols)

keys = [(x, y) for x in range(rows) for y in range(cols)]

np.array([[1, 2]]).shape

# create distributions
__sum_1 = img_1_array.sum()
# img_1_distribution = FiniteDistribution({rows * x + y: img_1_array[x][y]/__sum_1 for (x, y) in keys})
img_1_distribution = FiniteDistribution({(x, y): img_1_array[x][y]/__sum_1 for (x, y) in keys})

__sum_2 = img_2_array.sum()
# img_2_distribution = FiniteDistribution({rows * x + y: img_2_array[x][y]/__sum_2 for (x, y) in keys})
img_2_distribution = FiniteDistribution({(x, y): img_2_array[x][y]/__sum_2 for (x, y) in keys})

In [3]:
c = lambda x, y: (x[0] - y[0])**2 + (x[1] - y[1])**2

In [4]:
# p-norm
p = 2.
sinkhorn_runner = skern.get_pnorm_regularized_runner(p, c)
# entropic
sinkhorn_runner_entropic = skern.get_entropically_regularized_runner(c)


epsilon = 1.0
delta = 0.1

print("Running quadratic.")
pi_p, f_p, g_p, inner_p, outer_p = sinkhorn_runner.run_sinkhorn(img_1_distribution, img_2_distribution, epsilon, delta, dual_potential_precision_mult = 0.5)
print(f"Ran quadratic. Took {outer_p} outer iterations.")
# print("Running entropic.")
# pi_e, f_e, g_e, inner_e, outer_e = sinkhorn_runner_entropic.run_sinkhorn(img_1_distribution, img_2_distribution, epsilon, delta)
# print(f"Ran entropic. Took {outer_e} outer iterations.")

Running quadratic.


KeyboardInterrupt: 

In [ ]:
# interpolate two images geodesically
composite_img_array = np.zeros(img_1_array.shape)
for key in pi_p.keys():
    (x0, y0) = key[0]
    (x1, y1) = key[1] 
    composite_key = (0.5 * (x0 + x1), 0.5 * (y0 + y1))
    rounded_key = (int(composite_key[0] * rows), int(composite_key[1] * rows))
    composite_img_array[rounded_key[0], rounded_key[1]] += pi_p.get_probability(key)

composite_img = Image.fromarray(composite_img_array.astype(np.uint8))  # Convert to uint8 for image format

# Show the image
composite_img.show()

# Optionally, save the image
composite_img.save("images/composite_img_p.png")